In [ ]:
# 📦 Paso 1: Instalar dependencias
!pip install -q llama-cpp-python==0.2.67 gradio==4.26.0

# 🗃️ Paso 2: Montar Google Drive (opcional)
from google.colab import drive
use_gdrive = input("¿Deseas usar Google Drive para guardar modelos? (s/n): ").strip().lower() == 's'

if use_gdrive:
    drive.mount('/content/drive')
    base_path = "/content/drive/MyDrive/ModelosLLM/"
else:
    base_path = "/content/modelos_llm/"

import os
os.makedirs(base_path, exist_ok=True)

# 📥 Paso 3: Definir modelos disponibles
MODELOS = {
    "TinyLlama 1.1B Chat (INT4)": {
        "url": "https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf ",
        "nombre": "tinyllama.gguf"
    },
    "Phi-2 (2.7B INT4)": {
        "url": "https://huggingface.co/TheBloke/phi-2-GGUF/resolve/main/phi-2.Q4_K_M.gguf ",
        "nombre": "phi2.gguf"
    },
    "Mistral 7B Instruct (INT4)": {
        "url": "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf ",
        "nombre": "mistral.gguf"
    },
    "LLaMA2 7B Chat (INT4)": {
        "url": "https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf ",
        "nombre": "llama2.gguf"
    }
}

# 📦 Paso 4: Menú para elegir modelo y descargar
import urllib.request

print("Modelos disponibles:")
for i, k in enumerate(MODELOS):
    print(f"{i+1}. {k}")

opcion = int(input("Selecciona un modelo (número): ")) - 1
seleccion = list(MODELOS.keys())[opcion]
modelo_url = MODELOS[seleccion]["url"]
modelo_nombre = MODELOS[seleccion]["nombre"]
modelo_path = os.path.join(base_path, modelo_nombre)

# Descargar modelo si no existe
if not os.path.exists(modelo_path):
    print(f"Descargando {seleccion}...")
    urllib.request.urlretrieve(modelo_url, modelo_path)
else:
    print(f"Modelo ya descargado: {modelo_path}")

# 🤖 Paso 5: Cargar modelo con llama-cpp
from llama_cpp import Llama

llm = Llama(
    model_path=modelo_path,
    n_threads=8,
    n_ctx=2048,
    n_batch=512,
    verbose=False
)

# 🧠 Paso 6: Función de respuesta + interfaz Gradio
import gradio as gr

def responder(mensaje, historia):
    historia.append(("🧑", mensaje))
    prompt = "\n".join([f"Usuario: {x[0]}\nAsistente: {x[1]}" for x in historia[:-1]])
    prompt += f"\nUsuario: {mensaje}\nAsistente:"

    respuesta = llm(prompt, max_tokens=512, stop=["Usuario:", "Asistente:"])
    texto_respuesta = respuesta["choices"][0]["text"].strip()
    historia[-1] = (mensaje, texto_respuesta)
    return texto_respuesta, historia

iface = gr.ChatInterface(
    responder,
    chatbot=gr.Chatbot(),
    title="🧠 Chat con LLM (Nano style)",
    description=f"Modelo: {seleccion}",
    theme="soft",
)

iface.launch(share=True)